In [ ]:
import logging.config
import gradio as gr
import numpy as np
import os
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import logging
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
def chunk(docs):
    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(docs)
    logging.debug(f"{len(chunks)} chunks created.")
    return chunks

def load_file(file_path):
    # Load documents
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    logging.debug(f"{len(docs)} pages loaded.")
    return docs

def create_embeddings(chunks):
    embeddings = OllamaEmbeddings(model="llama3.2")
    # Create Embeddings
    db_name="vector_db"
    if os.path.exists(db_name):
        Chroma(persist_directory=db_name,embedding_function=embeddings).delete_collection()
    vectorstore = Chroma.from_documents(documents=chunks,embedding=embeddings, persist_directory=db_name)
    return vectorstore

def handle_file(file_path):
    docs =load_file(file_path)   
    chunks = chunk(docs)
    vectorstore = create_embeddings(chunks)
    return vectorstore
vectorstore = handle_file("../assets/AOS.pdf")

In [ ]:
result = vectorstore._collection.get(include=["embeddings","documents"])
documents = result["documents"]
vectors = np.array(result["embeddings"])

In [ ]:
vectors.shape

In [ ]:
model = ChatOllama(model="llama3.2",temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True) 
retriever = vectorstore.as_retriever(search_kwargs={"k":10})
conversation_chain = ConversationalRetrievalChain.from_llm(llm=model, retriever=retriever, memory=memory)

In [ ]:
def chat(message, hisotry):
    result = conversation_chain.invoke({"question":message})
    return result["answer"]

In [ ]:
view = gr.ChatInterface(chat).launch()